In [ ]:
from scipy.spatial import distance as dist
from keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
cnn=load_model('cnn.h5')
cap=cv2.VideoCapture(0)
i=0
while True:
    ret,frame=cap.read()
    faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=faceCascade.detectMultiScale(gray,1.1,4)
    cv2.putText(frame, str(len(faces))+" Face", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0) , 2, cv2.LINE_AA)
    D=0
    for x,y,w,h in faces:
        roi_gray=gray[y:y+h,x:x+w]
        roi_color=frame[y:y+h,x:x+w]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        stack_x = []
        stack_y = []
        D=0
        face_roi=roi_color[y:y+h,x:x+w]
        final_image = cv2.resize(roi_color,(64,64))
        test_image = image.img_to_array(final_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = cnn.predict(test_image)
        if result[0][0] == 0:
            prediction = 'No Mask'
            i=1
        else:
            prediction = 'Mask Detected'
            i=0
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[i],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[i],-1)
        cv2.putText(frame, prediction, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
            
    for i in range(0,len(faces)):
        x1 = faces[i][0]
        y1 = faces[i][1]
        x2 = faces[i][0] + faces[i][2]
        y2 = faces[i][1] + faces[i][3]

        mid_x = int((x1+x2)/2)
        mid_y = int((y1+y2)/2)
        stack_x.append(mid_x)
        stack_y.append(mid_y)
                
    if len(faces) == 2:
        D = int(dist.euclidean((stack_x.pop(), stack_y.pop()), (stack_x.pop(), stack_y.pop())))   
                
    if D<250 and D!=0:
        cv2.putText(frame, "Maintain Social Distancing", (100, 100), cv2.FONT_HERSHEY_SIMPLEX,1, [0,0,255] , 2)                
                
                
    cv2.imshow('LIVE',frame)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()